In [36]:
#  Ebnable HTML/CSS 
from IPython.core.display import HTML
HTML("<link href='https://fonts.googleapis.com/css?family=Passion+One' rel='stylesheet' type='text/css'><style>div.attn { font-family: 'Helvetica Neue'; font-size: 30px; line-height: 40px; color: #FFFFFF; text-align: center; margin: 30px 0; border-width: 10px 0; border-style: solid; border-color: #5AAAAA; padding: 30px 0; background-color: #DDDDFF; }hr { border: 0; background-color: #ffffff; border-top: 1px solid black; }hr.major { border-top: 10px solid #5AAA5A; }hr.minor { border: none; background-color: #ffffff; border-top: 5px dotted #CC3333; }div.bubble { width: 65%; padding: 20px; background: #DDDDDD; border-radius: 15px; margin: 0 auto; font-style: italic; color: #f00; }em { color: #AAA; }div.c1{visibility:hidden;margin:0;height:0;}div.note{color:red;}</style>")

___
Enter Team Member Names here (double click to edit):

- Ryan Bass
- Brett Benefield
- Cho Kim
- Nicole Wittlin

________

# In Class Assignment Three
In the following assignment you will be asked to fill in python code and derivations for a number of different problems. Please read all instructions carefully and turn in the rendered notebook (or HTML of the rendered notebook)  before the end of class.

<a id="top"></a>
## Contents
* <a href="#Loading">Loading the Data</a>
* <a href="#distance">Measuring Distances</a>
* <a href="#KNN">K-Nearest Neighbors</a>
* <a href="#naive">Naive Bayes</a>

________________________________________________________________________________________________________
<a id="Loading"></a>
<a href="#top">Back to Top</a>
## Downloading the Document Data
Please run the following code to read in the "20 newsgroups" dataset from sklearn's data loading module.

In [37]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import numpy as np
from __future__ import print_function

# this takes about 30 seconds to compute, read the next section while this downloads
ds = fetch_20newsgroups_vectorized(subset='train')

# this holds the continuous feature data (which is tfidf)
print('features shape:', ds.data.shape) # there are ~11000 instances and ~130k features per instance
print('target shape:', ds.target.shape) 
print('range of target:', np.min(ds.target),np.max(ds.target))
print('Data type is', type(ds.data), float(ds.data.nnz)/(ds.data.shape[0]*ds.data.shape[1])*100, '% of the data is non-zero')

features shape: (11314, 130107)
target shape: (11314,)
range of target: 0 19
Data type is <class 'scipy.sparse.csr.csr_matrix'> 0.1214353154362896 % of the data is non-zero


## Understanding the Dataset
Look at the description for the 20 newsgroups dataset at http://qwone.com/~jason/20Newsgroups/. You have just downloaded the "vectorized" version of the dataset, which means all the words inside the articles have gone through a transformation that binned them into 130 thousand features related to the words in them.  

**Question Set 1**:
- How many instances are in the dataset? 11,314
- What does each instance represent? News articles
- How many classes are in the dataset and what does each class represent? target represents different categories to classify into. 20 different classes. Categories that each of the documents would fall into.
- Would you expect a classifier trained on this data would generalize to documents written in the past week? Why or why not?
- Is the data represented as a sparse or dense matrix?
    * Sparse, less than 1% of the data of values greater than 0. 

___
Enter your answer here:
- How many instances are in the dataset?
    * 11,000 rows 

- What does each instance represent?
    * Our vocubulary is 130,000 unique words and every time we see one in an article, we increment the word count by 1. However, this is oversensitive to common words.  So we look at the frequency for each term and normalize this by the number of documents each word occurs in. 
  
- How many classes are in the dataset and what does each class represent? 
    * There are 20 classes in this dataset.
    * Each classes represents 20 subject types / genres from various news articles. 
    * Each instance is a seperate fully written article from the news.
  
- Is the data represented as a sparse or dense matrix?
    * This data was collected over 20 years ago. Words like iPhone, Obama, and Google may not be present of represented correctly in this data. So the classifier will work, but certain words will cause subject / genre classification challenges.





___
<a id="distance"></a>
<a href="#top">Back to Top</a>
## Measures of Distance
In the following block of code, we isolate three instances from the dataset. The instance "`a`" is from the group *computer graphics*, "`b`" is from from the group *recreation autos*, and "`c`" is from group *recreation motorcycle*. **Exercise for part 2**: Calculate the: 
- (1) Euclidean distance
- (2) Cosine distance 
- (3) Jaccard similarity 

between each pair of instances using the imported functions below. Remember that the Jaccard similarity is only for binary valued vectors, so convert vectors to binary using a threshold. 

**Question for part 2**: Which distance seems more appropriate to use for this data? **Why**?

In [38]:
from scipy.spatial.distance import cosine
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import jaccard
import numpy as np

# get first instance (comp)
idx = 550
a = ds.data[idx].todense()
a_class = ds.target_names[ds.target[idx]]
print('Instance A is from class', a_class)

# get second instance (autos)
idx = 4000
b = ds.data[idx].todense()
b_class = ds.target_names[ds.target[idx]]
print('Instance B is from class', b_class)

# get third instance (motorcycle)
idx = 7000
c = ds.data[idx].todense()
c_class = ds.target_names[ds.target[idx]]
print('Instance C is from class', c_class)

# Enter distance comparison below for each pair of vectors:
p = 'Placeholder'
print('\n\nEuclidean Distance\n ab:', euclidean(a,b), 'ac:', euclidean(a,c), 'bc:', euclidean(b,c))
print('Cosine Distance\n ab:', cosine(a,b), 'ac:', cosine(a,c), 'bc:', cosine(b,c))
print('Jaccard Dissimilarity (vectors should be boolean values)\n ab:', jaccard(a>0,b>0), 'ac:', jaccard(a>0,c>0), 'bc:', jaccard(b>0,c>0)
)

print('\n\nThe most appropriate distance is...')
print('Cosine. It clearly delineates between topics effectively. B and C are closest (by far)')
print('and A is about the same distance from B and C. Euclidean also works well, but not quite as good as "cosine"')

Instance A is from class comp.graphics
Instance B is from class rec.autos
Instance C is from class rec.motorcycles


Euclidean Distance
 ab: 1.0985184671870858 ac: 1.1891405425398234 bc: 0.9177794226661624
Cosine Distance
 ab: 0.6033714113755322 ac: 0.7070276149559529 bc: 0.4211595343347173
Jaccard Dissimilarity (vectors should be boolean values)
 ab: 0.8821138211382114 ac: 0.8754716981132076 bc: 0.9087947882736156


The most appropriate distance is...
Cosine. It clearly delineates between topics effectively. B and C are closest (by far)
and A is about the same distance from B and C. Euclidean also works well, but not quite as good as "cosine"


___
# Start of Live Session Assignment
___
<a id="KNN"></a>
<a href="#top">Back to Top</a>
## Using scikit-learn with KNN
Now let's use stratified cross validation with a holdout set to train a KNN model in `scikit-learn`. Use the example below to train a KNN classifier. The documentation for `KNeighborsClassifier` is here: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html  

**Exercise for part 3**: Use the code below to test what value of `n_neighbors` works best for the given data. *Note: do NOT change the metric to be anything other than `'euclidean'`. Other distance functions are not optimized for the amount of data we are working with.* 

**Question for part 3**: What is the accuracy of the best classifier you can create for this data (by changing only the `n_neighbors` parameter)? 

In [39]:
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from IPython.html import widgets 

cv = StratifiedShuffleSplit(ds.target, n_iter = 1, test_size = 0.5, train_size=0.5)

# fill in the training and testing data and save as separate variables
for trainidx, testidx in cv:
    # note that these are sparse matrices
    X_train = ds.data[trainidx] 
    X_test = ds.data[testidx] 
    y_train = ds.target[trainidx]
    y_test = ds.target[testidx]

# fill in your code  here to train and test
# calculate the accuracy and print it for various values of K
K = 1 
clf = KNeighborsClassifier(n_neighbors=K, weights='uniform', metric='euclidean')


acc = 0.2 # overwrite this with the actual accuracy

#=====================================

while K <= 10:
    clf = KNeighborsClassifier(n_neighbors=K, weights='uniform', metric='euclidean')
    clf.fit(X_train, y_train)
    
    ac = clf.score(X_test, y_test)
    print('Accuracy of classifier with %d neighbors is: %.2f'%(K,ac))
    K = K+1


Accuracy of classifier with 1 neighbors is: 0.60
Accuracy of classifier with 2 neighbors is: 0.55
Accuracy of classifier with 3 neighbors is: 0.52
Accuracy of classifier with 4 neighbors is: 0.51
Accuracy of classifier with 5 neighbors is: 0.50
Accuracy of classifier with 6 neighbors is: 0.49
Accuracy of classifier with 7 neighbors is: 0.48
Accuracy of classifier with 8 neighbors is: 0.47
Accuracy of classifier with 9 neighbors is: 0.46
Accuracy of classifier with 10 neighbors is: 0.45


The best accuracy is 60% with 1 neighbors.

**Question for part 3**: With sparse data, does the use of a KDTree representation make sense? Why or Why not?

Enter your answer below:

When there is sparse data, KDTrees have faster than normal query trees.

Ball tree and KD tree query times can be greatly influenced by data structure. In general, sparser data with a smaller intrinsic dimensionality leads to faster query times. Because the KD tree internal representation is aligned with the parameter axes, it will not generally show as much improvement as ball tree for arbitrarily structured data.
___

_____
## KNN extensions - Centroids
Now lets look at a very closely related classifier to KNN, called nearest centroid. In this classifier (which is more appropriate for big data scenarios and sparse data), the training step is used to calculate the centroids for each class. These centroids are saved. Unknown attributes, at prediction time, only need to have distances calculated for each saved centroid, drastically decreasing the time required for a prediction. 

**Exercise for part 4**: Use the template code below to create a nearest centroid classifier. Test which metric has the best cross validated performance: Euclidean, Cosine, or Manhattan. In `scikit-learn` you can see the documentation for NearestCentroid here: 
- http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestCentroid.html#sklearn.neighbors.NearestCentroid

and for supported distance metrics here:
- http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics

In [40]:
from sklearn.neighbors.nearest_centroid import NearestCentroid

# the parameters for the nearest centroid metric to test are:
#    l1, l2, and cosine (all are optimized)
metrics = ['euclidean', 'cosine', 'manhattan']

for m in metrics:
    clf = NearestCentroid(metric=m)
    clf.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    print("{} Accuracy: {}".format(m, acc))
    

print('Best one is cosine with 49.5% accuracy')

euclidean Accuracy: 0.41930351776560015
cosine Accuracy: 0.49142655117553474


/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/nearest_centroid.py:140: UserWarning: Averaging for metrics other than euclidean and manhattan not supported. The average is set to be the mean.
  warnings.warn("Averaging for metrics other than "


manhattan Accuracy: 0.1863178363089977
Best one is cosine with 49.5% accuracy


___
<a id="naive"></a>
<a href="#top">Back to Top</a>
## Naive Bayes Classification
Now let's look at the use of the Naive Bayes classifier. The 20 newsgroups dataset has 20 classes and about 130,000 features per instance. Recall that the Naive Bayes classifer calculates a posterior distribution for each possible class. Each posterior distribution is a multiplication of many conditional distributions: 

$${\arg \max}_{j} \left(p(class=j)\prod_{i} p(attribute=i|class=j) \right)$$

where $p(class=j)$ is the prior and $p(attribute=i|class=j)$ is the conditional probability.

**Question for part 5**: With this many classes and features, how many different conditional probabilities need to be parameterized? How many priors need to be parameterized?

Enter you answer here:

*Double Click to edit*


In [41]:
# Use this space for any calculations you might want to do
# Total conditionals = # of features / columns * number of classes should be roughly 2.6M
# There is one "prior" for each class

conditionals = 130107*20
print(conditionals)
print('20 priors need to be modeled')

2602140
20 priors need to be modeled


___
## Naive Bayes in Scikit-learn
Scikit has several implementations of the Naive Bayes classifier: `GaussianNB`, `MultinomialNB`, and `BernoulliNB`. Look at the documentation here: http://scikit-learn.org/stable/modules/naive_bayes.html Take a look at each implementation and then answer this question: 

**Questions for part 6**: 
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
- What if the data is sparse, does this change your answer? Why or Why not?

Enter you answer here:
- If the instances contain mostly continuous attributes, would it be better to use Gaussian Naive Bayes, Multinomial Naive Bayes, or Bernoulli? And Why? 
    * We would want to use a Gaussian Naive Bayes for mostly continuous attributes. Per the documentation, Bernoulli is best suited for binary classifications. Continuous attributes generally have a Gaussian distribution, whereas non continuous attributes do not have a Gaussian distribution.
- What if the data is sparse, does this change your answer? Why or Why not?
    * If the data is sparse it changes our answer because sparse matrices are much harder to find realistic Gaussian models because they have a mean near zero. It is better to use multinomial naive Bayes for sparse data. 
___

## Naive Bayes Comparison
For the final section of this notebook let's compare the performance of Naive Bayes for document classification. Look at the parameters for `MultinomialNB`, and `BernoulliNB` (especially `alpha` and `binarize`). 

**Exercise for part 7**: Using the example code below, change the parameters for each classifier and see how accurate you can make the classifiers on the test set. 

**Question for part 7**: Why are these implementations so fast to train? What does the `'alpha'` value control in these models (*i.e.*, how does it change the parameterizations)? 

* These implmentations are fast to train because we are dealing with very sparce matrices. 
* Since we are calculating the probability, in some probabilities the denominator can be zero and to avoid this we add in a very small alpha value. The best value of alpha depends on our dataset. 
* Increasing the binarize parameter decreases the accuracy of the model.
* The decoupling of the class conditional feature distributions means that each distribution can be independently estimated as a one dimensional distribution.

In [71]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB


clf_mnb = MultinomialNB(alpha=0.001)
clf_bnb = BernoulliNB(alpha=0.001, binarize=0.02)

# fill in your code here
alpha = np.arange(.001, 1, .05)
for a in alpha:
    clf = MultinomialNB(alpha=a)
    
    clf.fit(X_train, y_train)
    print("MultinomialNB Accuracy: {} for alpha {:.2f}".format(clf.score(X_test, y_test),a))

print()
print("For binarize = 0.02...")
alpha = np.arange(.001, .5, .05)
for a in alpha:
    clf = BernoulliNB(alpha=a, binarize = 0.02)
    
    clf.fit(X_train, y_train)
    print("BernoulliNB Accuracy: {} for alpha {:.2f}".format(clf.score(X_test, y_test),a))

print()
print("For binarize = 0.03...")
for a in alpha:
    clf = BernoulliNB(alpha=a, binarize = 0.03)
    
    clf.fit(X_train, y_train)
    print("BernoulliNB Accuracy: {} for alpha {:.2f}".format(clf.score(X_test, y_test),a))

print()
print("For binarize = 0.05...")
for a in alpha:
    clf = BernoulliNB(alpha=a, binarize = 0.05)
    
    clf.fit(X_train, y_train)
    print("BernoulliNB Accuracy: {} for alpha {:.2f}".format(clf.score(X_test, y_test),a))
    
print('These classifiers are so fast because...SEE ABOVE MARKDOWN CELL FOR RESPONSES')
print('The alpha values control...SEE ABOVE MARKDOWN CELL FOR RESPONSES')

MultinomialNB Accuracy: 0.88951741205586 for alpha 0.00
MultinomialNB Accuracy: 0.856107477461552 for alpha 0.05
MultinomialNB Accuracy: 0.8378999469683578 for alpha 0.10
MultinomialNB Accuracy: 0.824642036415061 for alpha 0.15
MultinomialNB Accuracy: 0.813328619409581 for alpha 0.20
MultinomialNB Accuracy: 0.8018384302633905 for alpha 0.25
MultinomialNB Accuracy: 0.7947675446349656 for alpha 0.30
MultinomialNB Accuracy: 0.7885805197100937 for alpha 0.35
MultinomialNB Accuracy: 0.7822167226445113 for alpha 0.40
MultinomialNB Accuracy: 0.7753226091567968 for alpha 0.45
MultinomialNB Accuracy: 0.7675446349655294 for alpha 0.50
MultinomialNB Accuracy: 0.7618879264627895 for alpha 0.55
MultinomialNB Accuracy: 0.7560544458193389 for alpha 0.60
MultinomialNB Accuracy: 0.7486300159094926 for alpha 0.65
MultinomialNB Accuracy: 0.7427965352660421 for alpha 0.70
MultinomialNB Accuracy: 0.7382004596075659 for alpha 0.75
MultinomialNB Accuracy: 0.7336043839490897 for alpha 0.80
MultinomialNB Accur

________________________________________________________________________________________________________

That's all! Please **upload your rendered notebook to blackboard** and please include **team member names** in the notebook submission.